In [1]:
%matplotlib inline
from scipy.stats import pearsonr,spearmanr
import pandas as pd
import matplotlib.pyplot as plt
import csv, os, datetime, traceback
from os.path import exists

import numpy as np
import tensorflow as tf
import seaborn as sns
from numpy import concatenate
from scipy import stats
from matplotlib import pyplot
from tensorflow import keras
from keras.models import Sequential, load_model, save_model
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.layers import Flatten
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from math import sqrt
import sys
# from keras_tuner import RandomSearch,BayesianOptimization
import IPython

ModuleNotFoundError: No module named 'matplotlib'

In [8]:
import random as rn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from tensorflow import keras
from keras.models import Sequential, load_model, save_model, Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, TimeDistributed, RepeatVector, Flatten, MaxPooling1D, Conv1D, Input, Embedding, Flatten, Concatenate

np.random.seed(37)
rn.seed(1254)
tf.random.set_seed(89)

def training():
    data = pd.read_csv("./Data/data.csv", sep = ',',encoding='utf-8', index_col=0)
    data.drop(columns = ['Product Group', 'SKU code', 'Event Code_plc', 'Event Code_mes', 'Work Center'], inplace = True)
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])

    data['hour_sin'] = np.sin(2 * np.pi * data['Timestamp'].dt.hour / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['Timestamp'].dt.hour / 24)
    data['day_of_week_sin'] = np.sin(2 * np.pi * data['Timestamp'].dt.dayofweek / 7)
    data['day_of_week_cos'] = np.cos(2 * np.pi * data['Timestamp'].dt.dayofweek / 7)
    data['month_sin'] = np.sin(2 * np.pi * data['Timestamp'].dt.month / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['Timestamp'].dt.month / 12)
    data['interval_sin'] = np.sin(2 * np.pi * data['Timestamp'].dt.minute // 15 / 4)
    data['interval_cos'] = np.cos(2 * np.pi * data['Timestamp'].dt.minute // 15 / 4)
    data = data.drop(columns = 'Timestamp')
    # Variables for training
    cols = list(data)
    df_for_training = data[cols]
    print(f'Shape of DataFrame: {df_for_training.shape}')

    # Fit scaler on training data and transform
    scaler = MinMaxScaler()
    X_numerical_scaled = scaler.fit_transform(df_for_training)   

    n_future = 1   # Number of 15-min we want to look into the future based on the past (Out).
    n_past = 16    # Number of past 15-min we want to use to predict the future (Step). Basically 4-hours
    features = len(cols) - 1  # Number of features

    def split_sequence(seq, steps, out):
        X, Y = list(), list()
        for i in range(len(seq)):
            end = i + steps
            outi = end + out
            if outi > len(seq)-1:
                break
            seqx, seqy = seq[i:end, 1:], seq[end:outi, 0] # 1 the column number I want to make the prediction
            X.append(seqx)
            Y.append(seqy)
        return np.array(X), np.array(Y)


    X_train, Y_train = split_sequence(X_numerical_scaled, n_past, n_future)
    X_train_temp, X_temp, y_train_temp, y_temp = train_test_split(X_train, Y_train, test_size=0.25, random_state=42, shuffle= True)
    X_val, X_test, Y_val, Y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle= True)
    X_train, Y_train = X_train_temp, y_train_temp
    print("Train sizes:")
    print(X_train.shape,Y_train.shape)
    print("Validation sizes:")
    print(X_val.shape,Y_val.shape)
    print("Test sizes:")
    print(X_test.shape,Y_test.shape)

    model = Sequential()
    model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], X_train.shape[2]))))
    model.add(Bidirectional(LSTM(512, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(1024, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(2048, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(1024, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(TimeDistributed(Dense(32, activation='relu')))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(1)))
    model.add(Flatten())
    model.add(Dense(n_future))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    history = model.fit(X_train, Y_train, epochs=10, batch_size=8, validation_data=(X_val, Y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=35)],
                        verbose=1, shuffle=True)

training()

Shape of DataFrame: (8543, 30)
Train sizes:
(6394, 16, 29) (6394, 1)
Validation sizes:
(1066, 16, 29) (1066, 1)
Test sizes:
(1066, 16, 29) (1066, 1)
Epoch 1/10


c:\Users\steli\.pyenv\pyenv-win\versions\3.12.1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 12/800 ━━━━━━━━━━━━━━━━━━━━ 1:12:27 6s/step - loss: 0.9995 - mse: 0.9995

KeyboardInterrupt: 